In [ ]:
import torch
from pyqcu.ascend import dslash
from pyqcu.ascend import inverse
dof = 12
# latt_size = (16, 16, 16, 16)
# latt_size = (4, 8, 8, 8)
latt_size = (8, 8, 8, 4)
# latt_size = (8, 8, 8, 8)
# latt_size = (4, 4, 4, 4)
kappa = 0.125
# dtype = torch.complex128
dtype = torch.complex64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
# Initialize lattice gauge theory
wilson = dslash.wilson_parity(
    latt_size=latt_size,
    kappa=kappa,
    dtype=dtype,
    device=device,
    verbose=False
)
clover = dslash.clover_parity(
    latt_size=latt_size,
    kappa=kappa,
    dtype=dtype,
    device=device,
    verbose=False
)
U = wilson.generate_gauge_field(sigma=0.1, seed=42)
null_vecs = torch.randn(dof, 4, 3, latt_size[3], latt_size[2], latt_size[1], latt_size[0],
                        dtype=dtype, device=device)
clover_term = clover.make_clover(U=U)
clover_term = clover.add_I(clover=clover_term)


b = torch.randn_like(null_vecs[0])
U_eo = dslash.xxxtzyx2pxxxtzyx(input_array=U)
clover_eo = dslash.xxxtzyx2pxxxtzyx(input_array=clover_term)


    @@@@@@######QCU NOTES START######@@@@@@@
    Guide:
    0. Required: MPI(e.g. 4.1.2), CUDA(e.g. 12.4), CMAKE(e.g. 3.22.1), GCC(e.g. 11.4.0), HDF5-MPI(e.g. 1.10.7,'apt install libhdf5-mpi-dev && export HDF5_MPI="ON" && pip install --no-binary=h5py h5py').
    1. The libqcu.so was compiled when pyqcu setup in download_path/PyQCU/lib, please add this path to your LD_LIBRARY_PATH.
    2. The QCU(PyQCU) splite grid by x->y->z->t, lattice by x->y->z->t->p->d->c->c or x->y->z->t->c->s(->p) and x->y->z->t->c->s->c->s(->p).
    3. The QUDA(PyQUDA) splite grid by t->z->y->x, lattice by c->c->x->y->z->t->p->d or c->s->x->y->z->t(->p) and c->s->c->s->x->y->z->t(->p).
    4. The QCU input params in numpy array(dtype=np.int32), argv in  numpy array(dtype=np.float32 or float64) array, set_ptrs in numpy array(dtype=np.int64), other in cupy array(dtype=cp.complex64 or complex128).
    5. The smallest lattice size is (wilson:x=4,y=4,z=4,t=4;clover:x=8,y=8,z=8,t=8) that QCU support (when '#define _B

In [2]:

mg = inverse.mg(b=b, wilson=wilson, U_eo=U_eo,
                clover=clover, clover_eo=clover_eo)


Building grid hierarchy:
  Level 0: 4x8x8x8
  Level 1: 2x4x4x4
  Level 2: 2x2x2x2
[[4, 8, 8, 8], [2, 4, 4, 4], [2, 2, 2, 2]]
@xxxtzyx2pxxxtzyx......
Splited Array Shape: torch.Size([2, 4, 3, 8, 8, 8, 2])
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@pxxxtzyx2xxxtzyx......
Restored Array Shape: torch.Size([4, 3, 8, 8, 8, 4])
@xxxtzyx2pxxxtzyx......
Splited Array Shape: torch.Size([2, 4, 3, 8, 8, 8, 2])
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@pxxxtzyx2xxxtzyx......
Restored Array Shape: torch.Size([4, 3, 8, 8, 8, 4])
@xxxtzyx2pxxxtzyx......
Splited Array Shape: torch.Size([2, 4, 3, 8, 8, 8, 2])
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@pxxxtzyx2xxxtzyx......
Restored Array Shape: torch.Size([4, 3, 8, 8, 8, 4])
@xxxtzyx2pxxxtzyx......
Splited Array Shape: torch.Size([2, 4, 3, 8, 8, 8, 2])
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@pxxxtzy

RuntimeError: einsum(): the number of subscripts in the equation (6) does not match the number of dimensions (10) for operand 1 and no ellipsis was given